<a href="https://colab.research.google.com/github/leohsuofnthu/Pytorch-IterativeFCN/blob/master/instance_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install SimpleITK


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 27 21:59:48 2019

@author: Gabriel Hsu
"""
import os
import torch
import numpy as np
from model import  iterativeFCN
from scipy import ndimage
import SimpleITK as sitk

#parameters
patch_size = 128

step = int(patch_size/2)
sigma_x = 2
lim_alternate_times = 20
n_min = 1000


img_name = './case1.mhd'
img = sitk.GetArrayFromImage(sitk.ReadImage(img_name))

#convert to HU
#test_numpy = test_numpy*1.0 - 1000.0

ins = np.zeros_like(img) 
mask = np.zeros_like(img)

temp = np.zeros_like(img)

print(img.shape)

img_shape = img.shape

def extract(img, x, y, z, patch_size):
    offset = int(patch_size/2)
    return img[z-offset:z+offset, y-offset:y+offset, x-offset:x+offset]

# Create FCN
print('Create Model!!')
model = iterativeFCN().to('cuda')
model.load_state_dict(torch.load('IterativeFCN_best_train_1e5.pth'))
print('Finish Loading Parameters!!')
  
  
#slide window with initial center coord
patch_size = 128
z = int(img.shape[0] - (patch_size/2))
y = int(patch_size/2)
x = int(patch_size/2)

#x_t-1, x_t
c_now = [z, y, x]
c_prev = [0, 0, 0]

label = 100
iters = 0
ii = 0
#slide window check
print('Start Instance Segmentation')
while z > 0:
  
  print(z, y, x)
  #extract patch and instance memory
  img_patch = torch.tensor(np.expand_dims(extract(img, x, y, z , 128), axis=0))
  ins_patch = torch.tensor(np.expand_dims(extract(ins, x, y, z, 128), axis=0))
  
  #sitk.WriteImage(sitk.GetImageFromArray(extract(img, x, y, z , 128)), './img18'+str(ii)+'.nrrd', True)
  #sitk.WriteImage(sitk.GetImageFromArray(extract(ins, x, y, z , 128)), './ins18'+str(ii)+'.nrrd', True)
  
  
  input_patch = torch.cat((img_patch, ins_patch))
  input_patch = torch.unsqueeze(input_patch, dim=0)
  
  with torch.no_grad():
      S, C = model(input_patch.float().to('cuda'))
  
  S = torch.squeeze(S.round().to('cpu')).numpy()
  
  """
  r = np.copy(torch.squeeze(img_patch.to('cpu')).numpy())
  r[ r < 1200] = 0
  r[r>0] = 1
  S = np.multiply(S , r)
  """
  
  vol = np.count_nonzero(S)
  sitk.WriteImage(sitk.GetImageFromArray(S), './gt18'+str(ii)+'.nrrd', True)
  
  ii+=1
  
  #check if vol > 1000
  if vol > n_min:
    c_prev[0] = c_now[0]
    c_prev[1] = c_now[1]
    c_prev[2] = c_now[2]
 
    
    center = ndimage.measurements.center_of_mass(S)
    print(center)
    center = [int(center[0]), int(center[1]), int(center[2])]
    print('Center relative to patch:', center)
    
    c_now[0] =  z +  (patch_size/2) - (patch_size - center[0])
    c_now[1] =  y - (patch_size/2) + center[1] 
    c_now[2] =  x - (patch_size/2) + center[2]
    print('Global Center:', c_now)
    
    #correction to be in-frame
    if (c_now[0] + patch_size/2) > img.shape[0]:
      c_now[0] = img.shape[0] - (patch_size/2)
        
    elif (c_now[0] - patch_size/2) < 0:
      c_now[0] = (patch_size/2)
      
    if (c_now[1] + patch_size/2) > img.shape[1]:
      c_now[1] = img.shape[1] - (patch_size/2)
    
    elif (c_now[1] - patch_size/2) < 0:
        c_now[1] = (patch_size/2)
      
    if (c_now[2] + patch_size/2) > img.shape[2]:
      c_now[2] = img.shape[2] - (patch_size/2)
      
    elif (c_now[2] - patch_size/2) < 0:
        c_now[0] = (patch_size/2)
      

    c_now[0] = int(c_now[0])
    c_now[1] = int(c_now[1])
    c_now[2] = int(c_now[2])
    print('modified center:', c_now)
    print('prev', c_prev)
    
    if abs(c_now[0]-c_prev[0]) > sigma_x or abs(c_now[1]-c_prev[1]) > sigma_x or abs(c_now[2]-c_prev[2]) > sigma_x:
      print('not converge')
      iters+=1
      print(iters)
      
      if iters == 20:
        print('iteration == 20')
        #pick avg and dim as converge
        c_now[0] = int((c_now[0] + c_prev[0])/2)
        c_now[1] = int((c_now[1] + c_prev[0])/2)
        c_now[2] = int((c_now[2] + c_prev[0])/2)
        
        print('converge and seg')
        iters=0
        #converge, update ins and mask
        
        
        z_low = int(c_now[0]-(patch_size/2))
        z_up = int(c_now[0]+(patch_size/2))
        y_low = int(c_now[1]-(patch_size/2))
        y_up = int(c_now[1]+(patch_size/2))
        x_low = int(c_now[2]-(patch_size/2))
        x_up = int(c_now[2]+(patch_size/2))
      
        r = S > 0
        ins[z_low:z_up, y_low:y_up, x_low:x_up][r] = 1 
        mask[z_low:z_up, y_low:y_up, x_low:x_up][r] = label
      
        label+=100
        print("seg {}th verts complete!!".format(label))
        #sitk.WriteImage(sitk.GetImageFromArray(ins), './ins18'+str(label)+'.nrrd', True)
        #sitk.WriteImage(sitk.GetImageFromArray(mask), './mask18'+str(label)+'.nrrd', True)
        
        
    else:
      print('converge and seg')
      iters=0
    
      #converge, update ins and mask
      z_low = int(c_now[0]-(patch_size/2))
      z_up = int(c_now[0]+(patch_size/2))
      y_low = int(c_now[1]-(patch_size/2))
      y_up = int(c_now[1]+(patch_size/2))
      x_low = int(c_now[2]-(patch_size/2))
      x_up = int(c_now[2]+(patch_size/2))
      
      r = S > 0
      ins[z_low:z_up, y_low:y_up, x_low:x_up][r] = 1 
      mask[z_low:z_up, y_low:y_up, x_low:x_up][r] = label
      
      sitk.WriteImage(sitk.GetImageFromArray(ins), './ins18'+str(label)+'.nrrd', True)
      sitk.WriteImage(sitk.GetImageFromArray(mask), './mask18'+str(label)+'.nrrd', True)
      
      label+=100
      print("seg {}th verts complete!!".format(label))
      
    #same patch analyze again, center remain
    z = c_now[0]
    y = c_now[1]
    x = c_now[2]
    
    
    
    
  else:
    print('slide window')
  #continue slide windows
    if x + step > img_shape[2]:
      x = int(patch_size/2)
      if y + step > img_shape[1]:
        y = int(patch_size/2)
        z = z - step
      else:
        y = y + step
    else:
      x = x + step
     

    print('Finish Segmentation!')



(404, 160, 160)
Create Model!!
num_channel 64
Finish Loading Parameters!!
Start Instance Segmentation
340 64 64
(120.88197523314479, 68.42880293533099, 80.47549304387708)
Center relative to patch: [120, 68, 80]
Global Center: [396.0, 68.0, 80.0]
modified center: [340, 68, 80]
prev [340, 64, 64]
not converge
1
340 68 80
(120.87758417481604, 63.409231357308954, 67.7954958111681)
Center relative to patch: [120, 63, 67]
Global Center: [396.0, 67.0, 83.0]
modified center: [340, 67, 83]
prev [340, 68, 80]
not converge
2
340 67 83
(121.07182320441989, 64.19694695493884, 65.23817583751158)
Center relative to patch: [121, 64, 65]
Global Center: [397.0, 67.0, 84.0]
modified center: [340, 67, 84]
prev [340, 67, 83]
converge and seg
seg 200th verts complete!!
340 67 84
(98.16776176540961, 61.1427493420205, 67.07560782581488)
Center relative to patch: [98, 61, 67]
Global Center: [374.0, 64.0, 87.0]
modified center: [340, 64, 87]
prev [340, 67, 84]
not converge
1
340 64 87
(98.180214287027, 64.14472

RuntimeError: ignored

In [0]:

sitk.WriteImage(sitk.GetImageFromArray(ins.numpy()), './ins18.nrrd', True)
sitk.WriteImage(sitk.GetImageFromArray(mask.numpy()), './mask18.nrrd', True)